In [2]:
import cv2
import numpy as np
import time

def process_frame(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)

    height, width = edges.shape[:2]
    mask = np.zeros_like(edges)
    region_of_interest_vertices = np.array([[(200, height), (500, 250), (800, height)]], dtype=np.int32)
    cv2.fillPoly(mask, region_of_interest_vertices, 255)
    masked_edges = cv2.bitwise_and(edges, mask)

    lines = cv2.HoughLinesP(masked_edges, 2, np.pi / 180, 100, np.array([]), minLineLength=40, maxLineGap=5)

    left_lines = []
    right_lines = []
    middle_lines = []

    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            slope = (y2 - y1) / (x2 - x1)
            intercept = y1 - slope * x1

            if slope < -0.1:
                left_lines.append((slope, intercept))
            elif slope > 0.2:
                right_lines.append((slope, intercept))
            else:
                middle_lines.append((x1, y1, x2, y2))

    line_image = np.zeros_like(frame)

    if left_lines:
        left_avg_slope, left_avg_intercept = np.average(left_lines, axis=0)
        draw_arrow(line_image, left_avg_slope, left_avg_intercept, color=(255, 0, 0))

    if right_lines:
        right_avg_slope, right_avg_intercept = np.average(right_lines, axis=0)
        draw_arrow(line_image, right_avg_slope, right_avg_intercept, color=(0, 0, 255))

    for line in middle_lines:
        x1, y1, x2, y2 = line
        cv2.line(line_image, (x1, y1), (x2, y2), (0, 255, 0), 5)

    result = cv2.addWeighted(frame, 0.8, line_image, 1, 1)
    return result

def draw_arrow(image, slope, intercept, color=(255, 0, 0), thickness=10, arrow_length=40, arrow_angle=np.pi / 6):
    y1 = image.shape[0]
    y2 = int(y1 * 3 / 5)
    x1 = int((y1 - intercept) / slope)
    x2 = int((y2 - intercept) / slope)

    cv2.line(image, (x1, y1), (x2, y2), color, thickness)
    angle = np.arctan2(y2 - y1, x2 - x1)

    x_arrow = int(x2 - arrow_length * np.cos(angle - arrow_angle))
    y_arrow = int(y2 - arrow_length * np.sin(angle - arrow_angle))
    cv2.line(image, (x2, y2), (x_arrow, y_arrow), color, thickness)

    x_arrow = int(x2 - arrow_length * np.cos(angle + arrow_angle))
    y_arrow = int(y2 - arrow_length * np.sin(angle + arrow_angle))
    cv2.line(image, (x2, y2), (x_arrow, y_arrow), color, thickness)

def calculate_risk_percentage(dat_file):
    data = np.genfromtxt(dat_file, delimiter=' ', skip_header=1, filling_values=np.nan)

    # Calculate risk score based on available data
    risk_score = np.nanmean(data[:, 5]) * 100

    return risk_score

def detect_drowsiness(frame, blink_count, last_blink_time):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rects = cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in rects:
        roi_gray = gray[y:y + h, x:x + w]
        eyes = eye_cascade.detectMultiScale(roi_gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        for (ex, ey, ew, eh) in eyes:
            eye_roi = roi_gray[ey: ey + eh, ex: ex + ew]
            eye = cv2.resize(eye_roi, (EAR_THRESH, EAR_THRESH))
            ear = eye_aspect_ratio(eye)

            # Check if the EAR falls below the threshold, indicating possible eye closure
            if ear < EAR_THRESHOLD:
                # Update blink count if the last blink occurred more than 0.5 seconds ago
                if time.time() - last_blink_time > 0.5:
                    blink_count += 1
                    last_blink_time = time.time()

    # Calculate drowsiness percentage based on blink count
    if blink_count >= 5:
        drowsiness_percentage = 100  # Return 100% if there are more than 5 blinks in 0.5 seconds
    else:
        drowsiness_percentage = 0

    return drowsiness_percentage


cap = cv2.VideoCapture("video.mp4")

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    # Resize the frame to match the new dimensions
    frame = cv2.resize(frame, (1000, 800))
    # Initialize face cascade classifier
    cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    # Initialize eye cascade classifier
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')



    # Show the frame
    cv2.imshow("Frame", frame)

    # Extract the road view frame
    road_view_frame = frame[:400, :500]  # Select the first part of upper half (road view)

    # Process the frame to detect lanes and draw arrows
    result = process_frame(road_view_frame)

    # Show the processed frame
    cv2.imshow("Processed Frame One", result)

    # Calculate risk percentage using the .dat file
    risk_percentage = calculate_risk_percentage("data.dat")
    

    # Detect drowsiness using the first frame of the lower half
    lower_half_frame = frame[400:, :500]
    drowsiness_percentage = detect_drowsiness(lower_half_frame, 5, 10)
    cv2.imshow('Processed Frame Two', lower_half_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

blink_count= 5
last_blink_time= 10
print("Risk Percentage:", risk_percentage)
print("Drowsiness Percentage:", drowsiness_percentage)
cap.release()
cv2.destroyAllWindows()


Risk Percentage: 982.621195618018
Drowsiness Percentage: 100
